### Домашнее задание
### Обучите нейронную сеть решать шифр цезаря.
####  Что надо сделать:
1. Написать алгоритм шифра цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
2. Сделать нейронную сеть
3. Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
4. Проверить качество

#### 1. Написать алгоритм шифра цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)

In [13]:
import pandas as pd
import time
import torch
from sklearn.model_selection import train_test_split

In [14]:
k='hellow'

In [16]:
def caesar(text, shift=2):
    LETTERS = 'abcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyz'
    result=[]
    for symbol in text:
        if symbol.isalpha():
            num = LETTERS.find(symbol)
            result.append(LETTERS[num+shift])
        else:
            result.append(symbol)
    
    return ''.join(result)


caesar('hellow2 kkz',2)

'jgnnqy2 mmb'

In [17]:
df = pd.read_csv('data.csv')
df.head(2)

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0


In [21]:
import re
regex = re.compile("[[a-zA-Z]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

In [30]:
#### почистим еще немного
df['clean']=df['normalized_text'].apply(words_only)

In [31]:
new_df=pd.DataFrame(df['clean'].dropna())

In [33]:
new_df.head(10)

,clean
0,maggie look whats that
1,lee mur lee mur
2,zee boo zee boo
3,im trying to teach maggie that nature doesnt e...
4,its like an ox only it has a hump and a dewlap...
5,you know his blood type how romantic
6,oh yeah whats my shoe size
7,ring
8,yes dad
9,ooh look maggie what is that do dec ah edron d...


In [35]:
new_df['caesar_text']=new_df.clean.apply(lambda x: caesar(str(x), 2))

In [44]:
new_df.head(10)

,clean,caesar_text
0,maggie look whats that,ociikg nqqm yjcvu vjcv
1,lee mur lee mur,ngg owt ngg owt
2,zee boo zee boo,bgg dqq bgg dqq
3,im trying to teach maggie that nature doesnt e...,ko vtakpi vq vgcej ociikg vjcv pcvwtg fqgupv g...
4,its like an ox only it has a hump and a dewlap...,kvu nkmg cp qz qpna kv jcu c jwor cpf c fgyncr...
5,you know his blood type how romantic,aqw mpqy jku dnqqf varg jqy tqocpvke
6,oh yeah whats my shoe size,qj agcj yjcvu oa ujqg ukbg
7,ring,tkpi
8,yes dad,agu fcf
9,ooh look maggie what is that do dec ah edron d...,qqj nqqm ociikg yjcv ku vjcv fq fge cj gftqp f...


In [40]:
## Оставим кусок для валидации
df_train, df_test = train_test_split(new_df, test_size=0.33)

In [48]:
text = [[c for c in ph] for ph in df_train.clean.tolist() if type(ph) is str]

In [49]:
ceas_text=[[caesar(c) for c in ph] for ph in text ]

In [55]:
CHARS = 'abcdefghijklmnopqrstuvwxyz '
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}
MAX_LEN = 50
CHAR_TO_INDEX

{'none': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 ' ': 27}

In [56]:
MAX_LEN = 50
Y = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        Y[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [57]:
MAX_LEN = 50
X= torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(ceas_text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [58]:
X[1]

tensor([22, 10,  3, 22, 21, 27,  5, 17, 20, 20,  7,  5, 22,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [59]:
Y[1]

tensor([20,  8,  1, 20, 19, 27,  3, 15, 18, 18,  5,  3, 20,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

#### 2. Сделать нейронную сеть

In [60]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        ## Здесь создать слои
        self.embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR),28)
        self.rnn = torch.nn.RNN(28,128, batch_first=True)
        self.linear = torch.nn.Linear(128,28)
        
    def forward(self, sentences, state=None):
        ## Здесь применить
        embds= self.embeddings(sentences)
        out, new_state = self.rnn(embds,state)
        result=self.linear(out)
        return result, new_state

In [61]:
model = Network()

In [62]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

#### 3. Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)

In [63]:
for ep in range(25):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        X_batch = X[i * 100:(i + 1) * 100]
        Y_batch = Y[i * 100:(i + 1) * 100].flatten()

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 3.321, Train loss: 1.443
Epoch 1. Time: 3.185, Train loss: 0.646
Epoch 2. Time: 3.135, Train loss: 0.363
Epoch 3. Time: 3.093, Train loss: 0.228
Epoch 4. Time: 3.263, Train loss: 0.157
Epoch 5. Time: 3.117, Train loss: 0.116
Epoch 6. Time: 3.183, Train loss: 0.089
Epoch 7. Time: 3.221, Train loss: 0.071
Epoch 8. Time: 3.073, Train loss: 0.059
Epoch 9. Time: 3.118, Train loss: 0.050
Epoch 10. Time: 3.231, Train loss: 0.043
Epoch 11. Time: 3.166, Train loss: 0.038
Epoch 12. Time: 3.011, Train loss: 0.034
Epoch 13. Time: 3.115, Train loss: 0.031
Epoch 14. Time: 3.123, Train loss: 0.028
Epoch 15. Time: 3.193, Train loss: 0.026
Epoch 16. Time: 3.131, Train loss: 0.024
Epoch 17. Time: 3.074, Train loss: 0.022
Epoch 18. Time: 3.190, Train loss: 0.021
Epoch 19. Time: 3.111, Train loss: 0.020
Epoch 20. Time: 3.037, Train loss: 0.018
Epoch 21. Time: 3.195, Train loss: 0.018
Epoch 22. Time: 3.122, Train loss: 0.017
Epoch 23. Time: 3.237, Train loss: 0.016
Epoch 24. Time: 3.181, Tra

In [75]:

def generate_sentence(sentence):
    print('caeser_text=>>',sentence)
    state = None
    new_sentence=[]
    for i in range( len(sentence) ):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long)
        result, state = model.forward( X, state )
        index = result.argmax()
        new_sentence.append(INDEX_TO_CHAR[index])
    print('encrypted text=>>',''.join(new_sentence))


In [76]:
## Посмотрим как оно работает
generate_sentence(sentence=caesar('hellow my dear friend'))

caeser_text=>> jgnnqy oa fgct htkgpf
encrypted text=>> hellow my dear friend


#### 4. Проверить качество

In [80]:
test.head()

,clean,caesar_text
909,yayyyyyy,acaaaaaa
475,please dont leave me george,rngcug fqpv ngcxg og igqtig
9854,are we truly in space and if so why,ctg yg vtwna kp urceg cpf kh uq yja
10671,he brought us gifts his first unselfish act,jg dtqwijv wu ikhvu jku hktuv wpugnhkuj cev
7064,mom bart has something to tell you,oqo dctv jcu uqogvjkpi vq vgnn aqw


In [88]:
text = [[c for c in ph] for ph in df_test.clean.tolist() if type(ph) is str]

In [79]:
ceas_text=[[caesar(c) for c in ph] for ph in text ]

In [91]:
MAX_LEN = 50
Y = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        Y[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [92]:
MAX_LEN = 50
X= torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(ceas_text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [ ]:
model.eval()

In [110]:
answers, _ = model.forward(X)

In [111]:
answers = answers.view(-1, len(INDEX_TO_CHAR))

In [135]:
## Точность на тесте
sum(answers.argmax(axis=1)==Y.flatten()).numpy()/Y.flatten().shape[0]

0.9985420463420984